## 1. Install gcloud CLI Tool

If you are running this notebook on GCP, the `gcloud` CLI tool is likely already pre-installed. However, if you are running into issue accessing `gcloud` command, first check your `$PATH` in the shell environment. If issue persists, you can use the following command to download the latest packages onto your environment.

In jupyter notebook cell:
```shell
!curl -sSL https://sdk.cloud.google.com | bash
!exec -l $SHELL
```

In terminal shell:

```shell
curl -sSL https://sdk.cloud.google.com | bash
exec -l $SHELL
```

#### 1.2 Update the gcloud component
We will use `gcloud components update` to update the ***gcloud*** command. This is not necessary if you are using Vertex AI workbench.



## 2. Setup API
We will perform the following action to setup the GenAI API for our python code use in the later portion.

### Pre-requisite:

You will need to add *Secret Manager* **manually** in the [IAM & Admin console](https://console.cloud.google.com/iam-admin/iam).


In [ ]:
!gcloud config list      # List your project ID & service account 

In [ ]:
import subprocess
import json
from time import sleep

# Key Variables
PROJECT_ID = "your-project-id"  # Replace with your GCP project ID
SECRET_NAME = "AIStudio"  # Secret name for Secret Manager
DISPLAY_NAME = "Generative Language API Key"  # Display name for the API key
API_SERVICE = ["https://aiplatform.googleapis.com",
               "https://generativelanguage.googleapis.com"]  # Service for API key restriction


def run_command(command, capture_output=True):
    """Run a shell command and handle errors."""
    try:
        result = subprocess.run(
            command,
            stdout=subprocess.PIPE if capture_output else None,
            stderr=subprocess.PIPE,
            text=True,
            shell=True,
        )
        if result.returncode != 0:
            raise subprocess.CalledProcessError(result.returncode, command, result.stderr)
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.cmd}")
        print(f"Error output: {e.stderr.strip()}")
        raise


def create_api_key(project_id, display_name):
    """Create or retrieve an API key."""
    print(f"Checking for existing API keys with display name: {display_name}")
    list_command = f"gcloud services api-keys list --project='{project_id}' --format=json"
    result = run_command(list_command)
    if result:
        api_keys = json.loads(result)
        matching_key = next((key for key in api_keys if key.get("displayName") == display_name), None)
        if matching_key:
            print(f"API key with display name '{display_name}' already exists.")
            api_key_name = matching_key.get("name")
            print("Note: KeyString cannot be retrieved for existing keys.")
            return None, api_key_name  # Return None for keyString since it can't be fetched for existing keys.

    print(f"Creating a new API key with display name: {display_name}")
    create_command = f"gcloud services api-keys create --display-name='{display_name}' --project='{project_id}' --format=json"
    result = run_command(create_command)
    if result:
        key_data = json.loads(result)
        response_data = key_data.get("response", {})
        api_key_name = response_data.get("name")
        api_key_string = response_data.get("keyString")
        if not api_key_name or not api_key_string:
            print("Failed to retrieve the API key name or key string from the response.")
            return None, None
        print(f"API key created successfully. Name: {api_key_name}")
        return api_key_string, api_key_name

    print("Failed to create or retrieve an API key.")
    return None, None


def secret_exists(secret_name, project_id):
    """Check if a secret exists in Secret Manager."""
    try:
        run_command(f"gcloud secrets describe {secret_name} --project='{project_id}'")
        return True
    except subprocess.CalledProcessError:
        return False


def store_in_secret_manager(secret_name, api_key, project_id):
    """Store the API key in Secret Manager."""
    import tempfile
    import os
    temp_file_path = None  # Initialize temp_file_path to avoid reference before assignment

    try:
        print(f"Creating secret '{secret_name}'...")
        create_secret_command = f"gcloud secrets create {secret_name} --replication-policy='automatic' --project='{project_id}'"
        run_command(create_secret_command)

        # Create a temporary file to store the secret data
        with tempfile.NamedTemporaryFile("w", delete=False) as temp_file:
            temp_file.write(api_key)
            temp_file_path = temp_file.name

        # Run the gcloud command to add the secret version
        command = (
            f"gcloud secrets versions add {secret_name} "
            f"--data-file={temp_file_path} "
            f"--project={project_id}"
        )
        result = subprocess.run(command, shell=True, check=True, text=True, stdout=subprocess.PIPE)
        print(f"Secret '{secret_name}' updated successfully.")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Failed to add secret version for '{secret_name}': {e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Clean up the temporary file if it was created
        if temp_file_path and os.path.exists(temp_file_path):
            os.unlink(temp_file_path)

    print(f"API key stored successfully in secret '{secret_name}'.")


def apply_restriction(api_key_name, api_service):
    """Apply API key restriction."""
    try:
        # Construct the --api-targets arguments
        api_targets = " ".join([f"--api-target='service={service}'" for service in api_service])

        # Construct and run the gcloud command
        command = (
            f"gcloud services api-keys update {api_key_name} "
            f"{api_targets} "
            f"--project='{PROJECT_ID}'"
        )
        run_command(command)
        print(f"Applied restrictions to API key '{api_key_name}' for services: {', '.join(api_service)}.")
    except Exception as e:
        print(f"Failed to apply restrictions to API key '{api_key_name}': {e}")

# Main workflow
print(f"Selected Project ID: {PROJECT_ID}")
print(f"Secret Name: {SECRET_NAME}")
print(f"API Service Restriction: {API_SERVICE}")

try:
    # Enable required services
    run_command(f"gcloud services enable apikeys.googleapis.com secretmanager.googleapis.com --project={PROJECT_ID}")

    # Create or retrieve API key
    api_key, api_key_name = create_api_key(PROJECT_ID, DISPLAY_NAME)
    if api_key_name:
        if api_key:
            print(f"New API key created: {api_key}")
            # Store API key in Secret Manager only if it's newly created
            store_in_secret_manager(SECRET_NAME, api_key, PROJECT_ID)
        else:
            print(f"Existing API key used: {api_key_name}")

        # Apply restriction to the API key
        apply_restriction(api_key_name, API_SERVICE)
        
        print("Note: It may take up to 5 minutes for settings to take effect")
    else:
        print("Failed to create or retrieve API key.")
except Exception as e:
    print(f"Error occurred during execution: {e}")

## Run below cell to install python library dependencies

In [ ]:
!pip install google-generativeai google-cloud-secret-manager

In [ ]:
import subprocess
import json
import pandas as pd
def get_projects_dataframe():
    # Command to get projects in JSON format
    command = ["gcloud", "projects", "list", "--format=json"]

    # Run the command and capture the output
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check for errors
    if result.returncode != 0:
        print("An error occurred while running gcloud projects list:")
        print(result.stderr)
        exit(1)

    # Parse the JSON output
    projects_data = json.loads(result.stdout)

    # Convert to pandas DataFrame
    df = pd.DataFrame(projects_data)

    return df

# Get the DataFrame
df = get_projects_dataframe()
# Key Variables
PROJECT_ID = "nigms-nosi-developers"  # Replace with your GCP project ID
SECRET_NAME = "Gemini-API"  # Secret name for Secret Manager
DISPLAY_NAME = "GeminiAPIKey"  # Display name for the API key
# Display the DataFrame
projectNumber = df['projectNumber']
secret_name = SECRET_NAME
version = "latest"
secret_path = f"projects/{projectNumber[0]}/secrets/{secret_name}/versions/{version}"
print(projectNumber[0])
print(secret_path)

**Now fetch your API key for Gemini from [this website](https://aistudio.google.com/app/apikey), click `Get API Key`.** and add it to your secret manager secret.

In [ ]:
from google.cloud import secretmanager

def access_secret_version(resource_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": resource_name})
    print(response)
    return response.payload.data.decode("UTF-8")


# Your secret's resource name
resource_name = f"projects/{projectNumber[0]}/secrets/{SECRET_NAME}/versions/{version}"

# Access the secret
apiKey = access_secret_version(resource_name)
print(resource_name)
print(apiKey)

In [ ]:
import vertexai
from vertexai.generative_models import (
    Content,
    Part,
    GenerationConfig,
    GenerativeModel,
)

# Initialize Vertex AI
PROJECT_ID = projectNumber[0]  # Replace with your project ID
vertexai.init(project=PROJECT_ID, location="us-central1")

# Initialize Gemini model
model = GenerativeModel("gemini-1.5-flash-002")

def explain(cell_number):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance

    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        content = (
            "Based on the input. Please concisely comment this code to explain each line. "
            "Ignore the run cell magic and just focus on the code or error: "
            + ipython.user_ns['In'][cell_number]
        )

        user_prompt_content = Content(
            role="user",
            parts=[
                Part.from_text(content),
            ],
        )

        response = model.generate_content(
            user_prompt_content,
            generation_config=GenerationConfig(temperature=0),
        )

        return display(Markdown(response.text))
    else:
        return "Cell number out of range."


def modify(cell_number, modification):
    """Modify the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance

    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        prompt = (
            "Please modify the code: " + ipython.user_ns['In'][cell_number] +
            " to accomplish " + modification +
            ". Ignore the run cell magic and just focus on the code. Assume all libraries are loaded."
        )

        user_prompt_content = Content(
            role="user",
            parts=[
                Part.from_text(prompt),
            ],
        )

        response = model.generate_content(
            user_prompt_content,
            generation_config=GenerationConfig(temperature=0),
        )

        return create_new_cell(response.text)
    else:
        return "Cell number out of range."


def propose(suggest):
    """Propose code based on the suggestion."""
    user_prompt_content = Content(
        role="user",
        parts=[
            Part.from_text(f"Please suggest code to accomplish: {suggest}. Return only code."),
        ],
    )

    response = model.generate_content(
        user_prompt_content,
        generation_config=GenerationConfig(temperature=0),
    )

    return create_new_cell(response.text)


def create_new_cell(contents):
    """Create a new cell in the Jupyter notebook."""
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)

In [ ]:
propose('write me code for openai')

In [ ]:
```python
import openai

# Set your OpenAI API key
openai.api_key = "YOUR_API_KEY"

def get_completion(prompt, model="text-davinci-003", max_tokens=100):
  response = openai.Completion.create(
    engine=model,
    prompt=prompt,
    max_tokens=max_tokens,
    n=1,
    stop=None,
    temperature=0.7,
  )
  return response.choices[0].text.strip()

# Example usage:
prompt = "Write a short story about a robot learning to love."
story = get_completion(prompt)
print(story)

```


In [ ]:
propose("I need to find the 25 most signficantly upregulated genes from a dataframe called gene. Here are the column names: ENTREZID, SYMBOL, GENENAME, logFC AveExpr, t, P.Value, adj.P.Val")

In [ ]:
propose("help me develop a nextflow workflow")